In [1]:
import torch
from torch_geometric.loader import DataLoader
from datasets.PowerFlowData import PowerFlowData

testset = PowerFlowData(root='data', case='14', split=[.5, .2, .3], task='test')
test_loader = DataLoader(testset, batch_size=128, shuffle=False)

c:\Users\nataxcan\miniconda3\envs\graphml_proj\lib\site-packages\torch_geometric\data\in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [4]:
# here you can experiment with the normalization function

def transform(data):
    ## selecting the right features
    # for x
    data.x[:,4] = data.x[:,4] - data.x[:,8] # Pd = Pd - Pg
    # + 3 for the one-hot encoding for four node types, -2 because we remove the index and Pg
    template = torch.zeros((data.x.shape[0], data.x.shape[1] + 3 - 2))
    template[:,0:4] = torch.nn.functional.one_hot(data.x[:,1].type(torch.int64))
    template[:,4:10] = data.x[:,2:8]
    data.x = template
    # for y
    # - 2 for removing index and type
    data.y = data.y[:,2:]

    # SHAPE NOW: torch.Size([14, 10]) torch.Size([14, 6]) for x and y

    ## normalizing
    # for node attributes
    xy = torch.concat([data.x[:,4:], data.y], dim=0)
    mean = torch.mean(xy, dim=0).unsqueeze(dim=0).expand(data.x.shape[0], 6)# 6 for:
    std = torch.std(xy, dim=0).unsqueeze(dim=0).expand(data.x.shape[0], 6)#   Vm, Va, Pd, Qd, Gs, Bs
    data.x[:,4:] = (data.x[:,4:] - mean) / (std + 0.1) # + 0.1 to avoid NaN's because of division by zero
    data.y = (data.y - mean) / (std + 0.1)
    # for edge attributes
    mean = torch.mean(data.edge_attr, dim=0).unsqueeze(dim=0).expand(data.edge_attr.shape[0], data.edge_attr.shape[1])
    std = torch.std(data.edge_attr, dim=0).unsqueeze(dim=0).expand(data.edge_attr.shape[0], data.edge_attr.shape[1])
    data.edge_attr = (data.edge_attr - mean) / (std + 0.1)

    ## adding the mask
    # where x and y are unequal, the network must predict
    unequal = (data.x[:,4:] != data.y).float()
    data.x = torch.concat([data.x, unequal], dim=1)

    return data

notnormed = PowerFlowData(root='data', case='14', split=[.5, .2, .3], task='test', normalize=False)
data1 = notnormed[90]
print(data1)
print(transform(data1))

# # print(xold.x[:,2:8]) # the old power values
# # print(xold.y[:,2].float())
# # print(xnew.x[:,4:10]) # the normalized ones
# # print(xnew.x[:,10:]) # the mask to know which 

Data(x=[14, 9], edge_index=[2, 20], edge_attr=[20, 5], y=[14, 8])
Data(x=[14, 16], edge_index=[2, 20], edge_attr=[20, 5], y=[14, 6])


In [3]:
# here you can see it in action

notnormed = PowerFlowData(root='data', case='14', split=[.5, .2, .3], task='test', normalize=False)
normed = PowerFlowData(root='data', case='14', split=[.5, .2, .3], task='test', normalize=True)
data1 = notnormed[90]
data2 = normed[90]

print(data1)
print(data2)

Data(x=[14, 9], edge_index=[2, 20], edge_attr=[20, 5], y=[14, 8])
Data(x=[14, 16], edge_index=[2, 20], edge_attr=[20, 5], y=[14, 6])
